# concatHSRLprelimData
This script combines multiple "preliminary_data" output files from the High Spectral Resolution Lidar (HSRL) into a single output netCDF file. These input data files were created in the field by HSRL scientists and include various corrections. The most comonly required variables for analysis are concatenated into single arrays, and the inclusion of additional variables is easy enough to do if desired.
<br><br>
_Written by Dan Stechman_
<br>
_University of Illinois at Urbana-Champaign_

In [ ]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr
from glob import glob
import time
import warnings
from datetime import datetime as dt

scriptStrtT = dt.now()

warnings.filterwarnings("ignore",category=RuntimeWarning)

## User Settings
This is where the most commonly modified settings reside (i.e., flight, input data path/filenames, output path/filename, etc.)

In [ ]:
# This variable should match the name of the parent directory containing
#    the flight data for the given mission. Used in defining file input 
#    and output names.
flight = 'RF01_20180116'

# Specify the location/name(s) of the input files and the output file
files = sorted(glob('/Volumes/SOCRATES_1/' + flight + '/HSRL/preliminary_data/*GVHSRL*.nc'))
outFileName = '/Volumes/SOCRATES_1/' + flight + '/HSRL/' + flight + '_concat-HSRL-prelimData.nc'

# These are the number of gates nearest the aircraft with negative gate range values (we throw these out)
#    **I've seen this number vary between projects, but seems consistent so far for SOCRATES at least
lGateStrt = 4

## Data File ID
Determine which data files will provide data between the user-defined start and end times

In [ ]:
# Create an array of file indices (from our file listing) to loop over
# lFileIx = np.arange(0,3)
lFileIx = np.arange(0,len(files))

## Define output variables
Loop through all of the input files and add up the total number of points in the time dimension and determine the number of gates (vertical dimension) after removing gates with negative ranges.
<br><br>
Then, create empty (zero-filled) arrays of the appropriate sizes for out output.

In [ ]:
print('{}    Determining output dimensions...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S')))
totalTime = 0;
for lFile in lFileIx:
    lidarData = xr.open_dataset(files[lFile],decode_times=False)
    totalTime += lidarData.dims['time']
totalRng = lidarData.dims['range'] - lGateStrt

print('\tTotal number of points in time dimension: {}'.format(totalTime))
print('\tTotal number of gates (in vertical dimension): {}'.format(totalRng))

In [ ]:
time1d_all = np.zeros((totalTime,))
telescopeDir_all = np.zeros((totalTime,))
gateAlt_all = np.zeros((totalTime,totalRng))
time_all = np.zeros((totalTime,totalRng))

AerosolBackscatterCoefficient_all = np.zeros((totalTime,totalRng))
AerosolBackscatterCoefficientVariance_all = np.zeros((totalTime,totalRng))
AerosolBackscatterCoefficientMask_all = np.zeros((totalTime,totalRng))
ParticleDepolarization_all = np.zeros((totalTime,totalRng))
ParticleDepolarizationVariance_all = np.zeros((totalTime,totalRng))
ParticleDepolarizationMask_all = np.zeros((totalTime,totalRng))
VolumeDepolarization_all = np.zeros((totalTime,totalRng))
VolumeDepolarizationVariance_all = np.zeros((totalTime,totalRng))
VolumeDepolarizationMask_all = np.zeros((totalTime,totalRng))
BackscatterRatio_all = np.zeros((totalTime,totalRng))
BackscatterRatioVariance_all = np.zeros((totalTime,totalRng))
BackscatterRatioMask_all = np.zeros((totalTime,totalRng))
MolecularBackscatterCoefficient_all = np.zeros((totalTime,totalRng))
MolecularBackscatterCoefficientVariance_all = np.zeros((totalTime,totalRng))
LowGainTotalBackscatterChannel_all = np.zeros((totalTime,totalRng))
LowGainTotalBackscatterChannelVariance_all = np.zeros((totalTime,totalRng))
# LowGainTotalBackscatterChannelMask_all = np.zeros((totalTime,totalRng))
HighGainTotalBackscatterChannel_all = np.zeros((totalTime,totalRng))
HighGainTotalBackscatterChannelVariance_all = np.zeros((totalTime,totalRng))
# HighGainTotalBackscatterChannelMask_all = np.zeros((totalTime,totalRng))
MolecularBackscatterChannel_all = np.zeros((totalTime,totalRng))
MolecularBackscatterChannelVariance_all = np.zeros((totalTime,totalRng))
CrossPolarizationChannel_all = np.zeros((totalTime,totalRng))
CrossPolarizationChannelVariance_all = np.zeros((totalTime,totalRng))
MergedCombinedChannel_all = np.zeros((totalTime,totalRng))
MergedCombinedChannelVariance_all = np.zeros((totalTime,totalRng))
MergedCombinedChannelMask_all = np.zeros((totalTime,totalRng))
AerosolExtinctionCoefficient_all = np.zeros((totalTime,totalRng))
AerosolExtinctionCoefficientVariance_all = np.zeros((totalTime,totalRng))
AerosolExtinctionCoefficientMask_all = np.zeros((totalTime,totalRng))

## Concatenate Variables Across All Files
Loop through the files covering the requested time period and fill out output arrays.
<br>
Also, create modified time and altitude arrays which need to be 2-dimensional for plotting.
<br>
> The altitude array is modified using the gate range from the aircraft to yield ground-relative altitudes for each gate.

In [ ]:
print('{}    Beginning file concatenation for {}...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S'),flight))
numFiles = len(lFileIx)
fileCount = 1
strtIx = 0
for lFile in lFileIx:
    print('\tAppending file {} of {}...'.format(fileCount,numFiles))
    
    fDate = dt.date(dt.strptime(files[lFile][-30:-22],'%Y%m%d')) # Get the current date as a date object
    
    # Open the current lidar data file
    lidarData = xr.open_dataset(files[lFile],decode_times=False)
    
    # Get length of time dimension of current file and define the end index
    #    of the array slice to place data within the output arrays
    endIx = lidarData.dims['time'] + strtIx
        
    # Pull out the dimension variables
    gateRange_1d = lidarData['range'].data[lGateStrt:]
    alt_1d = lidarData['GGALT'].data
    time_1d_hhmmss = [dt.time(dt.utcfromtimestamp(t_sec)) for t_sec in lidarData['time'].data]
    
    # Convert time to an array of timestamps (use the dt.from_timestamp() function when pulling back out)
    time_1d = [dt.timestamp(dt.combine(fDate, t)) for t in time_1d_hhmmss]
    
    # Get the telescope pointing direction
    telescopeDir = lidarData['TelescopeDirection'].data # 1 = upward, 0 = downward

    
    # Create arrays of gateRange and gate altitude matching dimensions of 2-D variables
    gateRange_2d = np.tile(gateRange_1d,(len(time_1d),1))
    time_2d = np.transpose(np.tile(time_1d,(len(gateRange_1d),1)))
    alt_2d = np.transpose(np.tile(alt_1d,(len(gateRange_1d),1)))
    
    gateRange_2d[np.where(telescopeDir == 1),:] *= -1

    # Add data from current file into slice of output arrays
    gateAlt_all[strtIx:endIx,:] = alt_2d - gateRange_2d # ground-relative altitude MSL of each gate
    time_all[strtIx:endIx,:] = time_2d
    
    time1d_all[strtIx:endIx] = time_1d[:]
    telescopeDir_all[strtIx:endIx] = telescopeDir[:]

    AerosolBackscatterCoefficient_all[strtIx:endIx,:] = lidarData['Aerosol_Backscatter_Coefficient'].data[:,lGateStrt:]
    AerosolBackscatterCoefficientVariance_all[strtIx:endIx,:] = lidarData['Aerosol_Backscatter_Coefficient_variance'].data[:,lGateStrt:]
    AerosolBackscatterCoefficientMask_all[strtIx:endIx,:] = lidarData['Aerosol_Backscatter_Coefficient_mask'].data[:,lGateStrt:]
    ParticleDepolarization_all[strtIx:endIx,:] = lidarData['Particle_Depolarization'].data[:,lGateStrt:]
    ParticleDepolarizationVariance_all[strtIx:endIx,:] = lidarData['Particle_Depolarization_variance'].data[:,lGateStrt:]
    ParticleDepolarizationMask_all[strtIx:endIx,:] = lidarData['Particle_Depolarization_mask'].data[:,lGateStrt:]
    VolumeDepolarization_all[strtIx:endIx,:] = lidarData['Volume_Depolarization'].data[:,lGateStrt:]
    VolumeDepolarizationVariance_all[strtIx:endIx,:] = lidarData['Volume_Depolarization_variance'].data[:,lGateStrt:]
    VolumeDepolarizationMask_all[strtIx:endIx,:] = lidarData['Volume_Depolarization_mask'].data[:,lGateStrt:]
    BackscatterRatio_all[strtIx:endIx,:] = lidarData['Backscatter_Ratio'].data[:,lGateStrt:]
    BackscatterRatioVariance_all[strtIx:endIx,:] = lidarData['Backscatter_Ratio_variance'].data[:,lGateStrt:]
    BackscatterRatioMask_all[strtIx:endIx,:] = lidarData['Backscatter_Ratio_mask'].data[:,lGateStrt:]
    MolecularBackscatterCoefficient_all[strtIx:endIx,:] = lidarData['Molecular_Backscatter_Coefficient'].data[:,lGateStrt:]
    MolecularBackscatterCoefficientVariance_all[strtIx:endIx,:] = lidarData['Molecular_Backscatter_Coefficient_variance'].data[:,lGateStrt:]
    LowGainTotalBackscatterChannel_all[strtIx:endIx,:] = lidarData['Low_Gain_Total_Backscatter_Channel'].data[:,lGateStrt:]
    LowGainTotalBackscatterChannelVariance_all[strtIx:endIx,:] = lidarData['Low_Gain_Total_Backscatter_Channel_variance'].data[:,lGateStrt:]
#     LowGainTotalBackscatterChannelMask_all[strtIx:endIx,:] = lidarData['Low_Gain_Total_Backscatter_Channel_mask'].data[:,lGateStrt:]
    HighGainTotalBackscatterChannel_all[strtIx:endIx,:] = lidarData['High_Gain_Total_Backscatter_Channel'].data[:,lGateStrt:]
    HighGainTotalBackscatterChannelVariance_all[strtIx:endIx,:] = lidarData['High_Gain_Total_Backscatter_Channel_variance'].data[:,lGateStrt:]
#     HighGainTotalBackscatterChannelMask_all[strtIx:endIx,:] = lidarData['High_Gain_Total_Backscatter_Channel_mask'].data[:,lGateStrt:]
    MolecularBackscatterChannel_all[strtIx:endIx,:] = lidarData['Molecular_Backscatter_Channel'].data[:,lGateStrt:]
    MolecularBackscatterChannelVariance_all[strtIx:endIx,:] = lidarData['Molecular_Backscatter_Channel_variance'].data[:,lGateStrt:]
    CrossPolarizationChannel_all[strtIx:endIx,:] = lidarData['Cross_Polarization_Channel'].data[:,lGateStrt:]
    CrossPolarizationChannelVariance_all[strtIx:endIx,:] = lidarData['Cross_Polarization_Channel_variance'].data[:,lGateStrt:]
    MergedCombinedChannel_all[strtIx:endIx,:] = lidarData['Merged_Combined_Channel'].data[:,lGateStrt:]
    MergedCombinedChannelVariance_all[strtIx:endIx,:] = lidarData['Merged_Combined_Channel_variance'].data[:,lGateStrt:]
    MergedCombinedChannelMask_all[strtIx:endIx,:] = lidarData['Merged_Combined_Channel_mask'].data[:,lGateStrt:]
    AerosolExtinctionCoefficient_all[strtIx:endIx,:] = lidarData['Aerosol_Extinction_Coefficient'].data[:,lGateStrt:]
    AerosolExtinctionCoefficientVariance_all[strtIx:endIx,:] = lidarData['Aerosol_Extinction_Coefficient_variance'].data[:,lGateStrt:]
    AerosolExtinctionCoefficientMask_all[strtIx:endIx,:] = lidarData['Aerosol_Extinction_Coefficient_mask'].data[:,lGateStrt:]

        
    # Move our starting index for the output array to immediately
    #    after the end of the current data slice
    strtIx = endIx
    
    fileCount += 1

## Write concatenated data out to NetCDF

In [ ]:
print('{}    Concatenation complete. Beginning netCDF write...'.format(dt.strftime(dt.now(),'%y/%m/%d %H:%M:%S')))
# Create a netCDF file to hold our output
rootGrp = Dataset(outFileName,'w',format='NETCDF4')
rootGrp.set_fill_on()

# Define the netCDF dimensions
gateRng = rootGrp.createDimension('gateRng',totalRng)
time1d = rootGrp.createDimension('time1d',totalTime)

# Create out output variable instances within the file and
#    define metadata for each as needed
TIME = rootGrp.createVariable('time1d','f8',('time1d',),fill_value=np.nan)
TIME.long_name = 'POSIX Timestamp'
TIME.units = 'Seconds since 1 Jan 1970'
TIME.coordinates = 'time1d'

GRNG = rootGrp.createVariable('gateRng','f4',('gateRng',),fill_value=np.nan)
GRNG.long_name = 'Distance from lidar to center of each range gate'
GRNG.units = 'm'
GRNG.coordinates = 'gateRng'

TIME2D = rootGrp.createVariable('time2d','f8',('time1d','gateRng'),fill_value=np.nan)
TIME2D.long_name = 'POSIX Timestamp for every gate/time'
TIME2D.units = 'Seconds since 1 Jan 1970'
TIME2D.coordinates = 'time1d gateRng'

GALT2D = rootGrp.createVariable('gateAlt2d','f4',('time1d','gateRng'),fill_value=np.nan)
GALT2D.long_name = 'Altitude of each gate above MSL'
GALT2D.units = 'm'
GALT2D.coordinates = 'time1d gateRng'

TELDIR = rootGrp.createVariable('TelescopeDir','i1',('time1d'))
TELDIR.description = 'Pointing direction of lidar telescope'
TELDIR.units = '1 = upward, 0 = downward'
TELDIR.coordinates = 'time1d'

###
AERO_BC = rootGrp.createVariable('Aerosol_Backscatter_Coefficient','f8',('time1d','gateRng'),fill_value=np.nan)
AERO_BC.units = 'm-1 sr-1'
AERO_BC.description = 'Calibrated Measurement of Aerosol Backscatter Coefficient in m-1 sr-1'
AERO_BC.coordinates = 'time1d gateRng'

AERO_BC_V = rootGrp.createVariable('Aerosol_Backscatter_Coefficient_variance','f4',('time1d','gateRng'),fill_value=np.nan)
AERO_BC_V.coordinates = 'time1d gateRng'

AERO_BC_M = rootGrp.createVariable('Aerosol_Backscatter_Coefficient_mask','i1',('time1d','gateRng'))
AERO_BC_M.units = '1 = Masked, 0 = Not Masked'
AERO_BC_M.coordinates = 'time1d gateRng'


###
PART_DEPOL = rootGrp.createVariable('Particle_Depolarization','f8',('time1d','gateRng'),fill_value=np.nan)
PART_DEPOL.units = 'unitless'
PART_DEPOL.description = ('Propensity of Particles to depolarize (d).  This is not identical to the depolarization ratio.' 
                          'See Gimmestad: 10.1364/AO.47.003795 or Hayman and Thayer: 10.1364/JOSAA.29.000400')
PART_DEPOL.coordinates = 'time1d gateRng'

PART_DEPOL_V = rootGrp.createVariable('Particle_Depolarization_variance','f8',('time1d','gateRng'),fill_value=np.nan)
PART_DEPOL_V.coordinates = 'time1d gateRng'

PART_DEPOL_M = rootGrp.createVariable('Particle_Depolarization_mask','i1',('time1d','gateRng'))
PART_DEPOL_M.units = '1 = Masked, 0 = Not Masked'
PART_DEPOL_M.coordinates = 'time1d gateRng'


###
VOL_DEPOL = rootGrp.createVariable('Volume_Depolarization','f8',('time1d','gateRng'),fill_value=np.nan)
VOL_DEPOL.units = 'unitless'
VOL_DEPOL.description = ('Propensity of Volume to depolarize (d).  This is not identical to the depolarization ratio.' 
                          'See Gimmestad: 10.1364/AO.47.003795 or Hayman and Thayer: 10.1364/JOSAA.29.000400')
VOL_DEPOL.coordinates = 'time1d gateRng'

VOL_DEPOL_V = rootGrp.createVariable('Volume_Depolarization_variance','f8',('time1d','gateRng'),fill_value=np.nan)
VOL_DEPOL_V.coordinates = 'time1d gateRng'

VOL_DEPOL_M = rootGrp.createVariable('Volume_Depolarization_mask','i1',('time1d','gateRng'))
VOL_DEPOL_M.units = '1 = Masked, 0 = Not Masked'
VOL_DEPOL_M.coordinates = 'time1d gateRng'


###
BS_RATIO = rootGrp.createVariable('Backscatter_Ratio','f8',('time1d','gateRng'),fill_value=np.nan)
BS_RATIO.units = 'unitless'
BS_RATIO.description = 'Ratio of combined to molecular backscatter'
BS_RATIO.coordinates = 'time1d gateRng'

BS_RATIO_V = rootGrp.createVariable('Backscatter_Ratio_variance','f8',('time1d','gateRng'),fill_value=np.nan)
BS_RATIO_V.coordinates = 'time1d gateRng'

BS_RATIO_M = rootGrp.createVariable('Backscatter_Ratio_mask','i1',('time1d','gateRng'))
BS_RATIO_M.units = '1 = Masked, 0 = Not Masked'
BS_RATIO_M.coordinates = 'time1d gateRng'


###
MOLECBS_COEF = rootGrp.createVariable('Molecular_Backscatter_Coefficient','f8',('time1d','gateRng'),fill_value=np.nan)
MOLECBS_COEF.units = 'm-1 sr-1'
MOLECBS_COEF.description = 'Ideal Atmosphere Molecular Backscatter Coefficient in m-1 sr-1'
MOLECBS_COEF.coordinates = 'time1d gateRng'

MOLECBS_COEF_V = rootGrp.createVariable('Molecular_Backscatter_Coefficient_variance','f8',('time1d','gateRng'),fill_value=np.nan)
MOLECBS_COEF_V.coordinates = 'time1d gateRng'


###
LOWGAIN_TBC = rootGrp.createVariable('Low_Gain_Total_Backscatter_Channel','f8',('time1d','gateRng'),fill_value=np.nan)
LOWGAIN_TBC.units = 'Photon Counts'
LOWGAIN_TBC.description = 'Parallel Polarization, Low Gain, Combined Aerosol and Molecular Returns'
LOWGAIN_TBC.coordinates = 'time1d gateRng'

LOWGAIN_TBC_V = rootGrp.createVariable('Low_Gain_Total_Backscatter_Channel_variance','f8',('time1d','gateRng'),fill_value=np.nan)
LOWGAIN_TBC_V.coordinates = 'time1d gateRng'


###
HIGAIN_TBC = rootGrp.createVariable('High_Gain_Total_Backscatter_Channel','f8',('time1d','gateRng'),fill_value=np.nan)
HIGAIN_TBC.units = 'Photon Counts'
HIGAIN_TBC.description = 'Parallel Polarization, High Gain, Combined Aerosol and Molecular Returns'
HIGAIN_TBC.coordinates = 'time1d gateRng'

HIGAIN_TBC_V = rootGrp.createVariable('High_Gain_Total_Backscatter_Channel_variance','f8',('time1d','gateRng'),fill_value=np.nan)
HIGAIN_TBC_V.coordinates = 'time1d gateRng'


###
MOLEC_BSC = rootGrp.createVariable('Molecular_Backscatter_Channel','f8',('time1d','gateRng'),fill_value=np.nan)
MOLEC_BSC.units = 'Photon Counts'
MOLEC_BSC.description = 'Parallel Polarization\nMolecular Backscatter Returns'
MOLEC_BSC.coordinates = 'time1d gateRng'

MOLEC_BSC_V = rootGrp.createVariable('Molecular_Backscatter_Channel_variance','f8',('time1d','gateRng'),fill_value=np.nan)
MOLEC_BSC_V.coordinates = 'time1d gateRng'


###
CRSPOL = rootGrp.createVariable('Cross_Polarization_Channel','f8',('time1d','gateRng'),fill_value=np.nan)
CRSPOL.units = 'Photon Counts'
CRSPOL.description = 'Cross Polarization\nCombined Aerosol and Molecular Returns'
CRSPOL.coordinates = 'time1d gateRng'

CRSPOL_V = rootGrp.createVariable('Cross_Polarization_Channel_variance','f8',('time1d','gateRng'),fill_value=np.nan)
CRSPOL_V.coordinates = 'time1d gateRng'


###
MRG_CMBND = rootGrp.createVariable('Merged_Combined_Channel','f8',('time1d','gateRng'),fill_value=np.nan)
MRG_CMBND.units = 'Photon Counts'
MRG_CMBND.description = 'Merged hi/lo gain combined channel'
MRG_CMBND.coordinates = 'time1d gateRng'

MRG_CMBND_V = rootGrp.createVariable('Merged_Combined_Channel_variance','f8',('time1d','gateRng'),fill_value=np.nan)
MRG_CMBND_V.coordinates = 'time1d gateRng'

MRG_CMBND_M = rootGrp.createVariable('Merged_Combined_Channel_mask','i1',('time1d','gateRng'))
MRG_CMBND_M.units = '1 = Masked, 0 = Not Masked'
MRG_CMBND_M.coordinates = 'time1d gateRng'


###
AERO_EXT_COEF = rootGrp.createVariable('Aersol_Extinction_Coefficient','f8',('time1d','gateRng'),fill_value=np.nan)
AERO_EXT_COEF.units = 'm-1'
AERO_EXT_COEF.description = 'Aerosol Extinction Coefficient'
AERO_EXT_COEF.coordinates = 'time1d gateRng'

AERO_EXT_COEF_V = rootGrp.createVariable('Aersol_Extinction_Coefficient_variance','f8',('time1d','gateRng'),fill_value=np.nan)
AERO_EXT_COEF_V.coordinates = 'time1d gateRng'

AERO_EXT_COEF_M = rootGrp.createVariable('Aersol_Extinction_Coefficient_mask','i1',('time1d','gateRng'))
AERO_EXT_COEF_M.units = '1 = Masked, 0 = Not Masked'
AERO_EXT_COEF_M.coordinates = 'time1d gateRng'




# Define global attributes
rootGrp.description = 'Concatenated High Spectral Resolution Lidar preliminary_data data'
rootGrp.flight = flight
rootGrp.history = 'Created ' + time.asctime(time.gmtime()) + ' UTC'
rootGrp.firstFile = files[lFileIx[0]]
rootGrp.lastFile = files[lFileIx[-1]]
rootGrp.lidarWavelength = 5.32e-07
rootGrp.ProcessingStatus= ('Raw Data,Removed specified times,Applied Pointwise Mask,'
                           'Nonlinear CountRate Correction for dead time 28.4 ns,Background Subtracted over [28952.46, 29694.44] m,' 
                           'Grab Range Slice 0.0 - 13874.6 m,Converted range to altitude data,'
                           'Applied Pointwise Mask,Time Resampled to dt= 2.0 s,Performed piecewise multiplication,'
                           'Profile Rescaled by array betwen 0.687502 and 0.687502,'
                           'Copy of previous profile: Molecular Backscatter Channel,'
                           'Multiplied by Molecular Backscatter Coefficient,Applied Pointwise Mask,Applied Pointwise Mask')

## Write data into netCDF variables
TIME[:] = time1d_all
GRNG[:] = gateRange_1d
TIME2D[:] = time_all
GALT2D[:] = gateAlt_all
TELDIR[:] = telescopeDir_all
AERO_BC[:] = AerosolBackscatterCoefficient_all
AERO_BC_V[:] = AerosolBackscatterCoefficientVariance_all
AERO_BC_M[:] = AerosolBackscatterCoefficientMask_all
PART_DEPOL[:] = ParticleDepolarization_all
PART_DEPOL_V[:] = ParticleDepolarizationVariance_all
PART_DEPOL_M[:] = ParticleDepolarizationMask_all
VOL_DEPOL[:] = VolumeDepolarization_all
VOL_DEPOL_V[:] = VolumeDepolarizationVariance_all
VOL_DEPOL_M[:] = VolumeDepolarizationMask_all
BS_RATIO[:] = BackscatterRatio_all
BS_RATIO_V[:] = BackscatterRatioVariance_all
BS_RATIO_M[:] = BackscatterRatioMask_all
MOLECBS_COEF[:] = MolecularBackscatterCoefficient_all
MOLECBS_COEF_V[:] = MolecularBackscatterCoefficientVariance_all
LOWGAIN_TBC[:] = LowGainTotalBackscatterChannel_all
LOWGAIN_TBC_V[:] = LowGainTotalBackscatterChannelVariance_all
HIGAIN_TBC[:] = HighGainTotalBackscatterChannel_all
HIGAIN_TBC_V[:] = HighGainTotalBackscatterChannelVariance_all
MOLEC_BSC[:] = MolecularBackscatterChannel_all
MOLEC_BSC_V[:] = MolecularBackscatterChannelVariance_all
CRSPOL[:] = CrossPolarizationChannel_all
CRSPOL_V[:] = CrossPolarizationChannelVariance_all
MRG_CMBND[:] = MergedCombinedChannel_all
MRG_CMBND_V[:] = MergedCombinedChannelVariance_all
MRG_CMBND_M[:] = MergedCombinedChannelMask_all
AERO_EXT_COEF[:] = AerosolExtinctionCoefficient_all
AERO_EXT_COEF_V[:] = AerosolExtinctionCoefficientVariance_all
AERO_EXT_COEF_M[:] = AerosolExtinctionCoefficientMask_all



# Close the output file
rootGrp.close()

print('\nTotal script run time: {}'.format(dt.now() - scriptStrtT))